In [1]:
import os
import sys
os.chdir('../')
sys.path.append(os.path.join(os.getcwd(), "src"))

In [2]:
from dataclasses import dataclass
from pathlib import Path
from WattPredictor.utils.helpers import *
from WattPredictor.utils.exception import *
from WattPredictor.constants import *
from WattPredictor import logger

In [3]:
@dataclass
class ModelTrainerConfig:
    root_dir: Path
    model_name: str
    train_features: Path
    test_features: Path
    x_transform: Path
    y_transform: Path
    input_seq_len: int
    step_size: int
    n_trials: int

@dataclass(frozen=True)
class FeatureStoreConfig:
    hopsworks_project_name: str
    hopsworks_api_key: str

In [4]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_PATH,
                       params_filepath=PARAMS_PATH,
                       schema_filepath=SCHEMA_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.model_trainer
        trans = self.params.transformation

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=Path(config.root_dir),
            train_features= Path(config.train_features),
            test_features= Path(config.test_features),
            x_transform= Path(config.x_transform),
            y_transform= Path(config.y_transform),
            model_name=config.model_name,
            input_seq_len= trans.input_seq_len,
            step_size = trans.step_size,
            n_trials=params.n_trials
        )

        return model_trainer_config
    

    def get_feature_store_config(self) -> FeatureStoreConfig:

        config = self.config.feature_store

        feature_store_config = FeatureStoreConfig(
                hopsworks_project_name=config.hopsworks_project_name,
                hopsworks_api_key=os.environ['hopsworks_api_key'],
        )

        return feature_store_config

In [5]:
import hopsworks
import pandas as pd
import sys
import os
from WattPredictor.utils.exception import CustomException
from WattPredictor import logger

class FeatureStore:
    def __init__(self, config):
        try:
            self.config = config
            self.connect()
        except Exception as e:
            raise CustomException(e, sys)

    def connect(self):
        try:
            self.project = hopsworks.login(
                project=self.config.hopsworks_project_name,
                api_key_value=self.config.hopsworks_api_key
            )
            self.feature_store = self.project.get_feature_store()
            self.dataset_api = self.project.get_dataset_api()
            logger.info(f"Connected to Hopsworks Feature Store: {self.config.hopsworks_project_name}")
        except Exception as e:
            raise CustomException(e, sys)

    def create_feature_group(self, name, df, primary_key, event_time, description):
        try:
            try:
                fg = self.feature_store.get_feature_group(name=name, version=1)
                logger.info(f"Feature Group '{name}' already exists. Inserting data instead.")
                fg.insert(df)
            except:
                logger.info(f"Feature Group '{name}' does not exist. Creating new one.")
                fg = self.feature_store.get_or_create_feature_group(
                    name=name,
                    version=1,
                    primary_key=primary_key,
                    event_time=event_time,
                    description=description,
                    online_enabled=False
                )
                fg.save(df)

            logger.info(f"Feature Group '{name}' created/updated successfully")

        except Exception as e:
            raise CustomException(e, sys)

    def create_feature_view(self, name: str, feature_group_name: str, features: list):
        try:
            fg = self.feature_store.get_feature_group(name=feature_group_name, version=1)
            fv = self.feature_store.get_or_create_feature_view(
                name=name,
                version=1,
                query=fg.select(features),
                description=f"Feature View for {name}"
            )
            logger.info(f"Feature View '{name}' created successfully")
        except Exception as e:
            raise CustomException(e, sys)

    def upload_file_safely(self, local_path: str, target_name: str):
        """
        Upload file to Hopsworks dataset storage.
        If it already exists, it will be overwritten.
        """
        try:
            self.dataset_api.upload(
                local_path,
                f"Resources/wattpredictor_artifacts/{target_name}",
                overwrite=True 
            )
            logger.info(f"Uploaded file to Feature Store: {target_name}")
        except Exception as e:
            raise CustomException(e, sys)
        
    def download_file(self, remote_name: str, local_path: str = None):
        """
        Download a file from Hopsworks dataset storage.

        Args:
            remote_name: filename in Hopsworks (inside wattpredictor_artifacts)
            local_path: optional local path to save the file. If None, saves in current directory.
        """
        try:
            target_path = f"Resources/wattpredictor_artifacts/{remote_name}"
            if local_path is None:
                local_path = remote_name

            self.dataset_api.download(
                target_path,
                local_path=local_path,
                overwrite=True
            )
            logger.info(f"Downloaded file from Feature Store: {remote_name} to {local_path}")

        except Exception as e:
            raise CustomException(e, sys)


    def delete_file(self, target_name: str):
        """
        Delete file from Hopsworks dataset storage.
        Only use this if you want to clean up files manually.
        """
        try:
            full_path = f"Resources/wattpredictor_artifacts/{target_name}"
            self.dataset_api.delete(full_path)
            logger.warning(f"Deleted file from Feature Store: {target_name}")
        except Exception as e:
            logger.warning(f"File not found or already deleted: {target_name}")
            # Not raising exception here to allow safe cleanup

    def get_training_data(self, feature_view_name: str):
        try:
            fv = self.feature_store.get_feature_view(name=feature_view_name, version=1)
            X, y = fv.training_data()
            logger.info(f"Retrieved training data from Feature View '{feature_view_name}'")
            return X, y
        except Exception as e:
            raise CustomException(e, sys)

In [6]:
import os
import sys
import mlflow
import optuna
from tqdm import tqdm
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from datetime import datetime
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error,root_mean_squared_error
from WattPredictor.utils.helpers import create_directories
from WattPredictor.utils.exception import CustomException
from WattPredictor import logger
from sklearn.preprocessing import StandardScaler
import xgboost
import lightgbm


class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig, feature_store_config):
        self.config = config
        self.feature_store_config = feature_store_config
        self.feature_store = FeatureStore(feature_store_config)

        mlflow.set_tracking_uri("file:./mlruns")
        mlflow.set_experiment("Electricity Demand Prediction")

        logger.info("MLflow tracking setup complete.")


        self.models = {
            "XGBoost": {
                "class": XGBRegressor,
                "search_space": lambda trial: {
                    "n_estimators": trial.suggest_int("n_estimators", 50, 300),
                    "max_depth": trial.suggest_int("max_depth", 3, 10),
                    "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
                },
                "mlflow_module": mlflow.xgboost,
            },
            "LightGBM": {
                "class": LGBMRegressor,
                "search_space": lambda trial: {
                    "num_leaves": trial.suggest_int("num_leaves", 20, 150),
                    "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
                    "n_estimators": trial.suggest_int("n_estimators", 50, 300),
                },
                "mlflow_module": mlflow.lightgbm,
            },
        }

    def load_inputs(self):
        try:
            self.feature_store.dataset_api.download("Resources/wattpredictor_artifacts/train_df.csv/train_features.csv", overwrite=True)
            self.feature_store.dataset_api.download("Resources/wattpredictor_artifacts/test_df.csv/test_features.csv", overwrite=True)
            train_df = pd.read_csv(self.config.train_features)
            test_df = pd.read_csv(self.config.test_features)

            return train_df, test_df

        except Exception as e:
            raise CustomException(e, sys)

    def get_cutoff_indices(self, df: pd.DataFrame, input_seq_len: int, step_size: int):
        stop = len(df) - input_seq_len - 1
        return [(i, i + input_seq_len, i + input_seq_len + 1) for i in range(0, stop, step_size)]
        
    def generate_ts_features_and_target(self, ts_data: pd.DataFrame):
            
            assert set(['date', 'demand', 'sub_region_code', 'temperature_2m']).issubset(ts_data.columns)

            region_codes = ts_data['sub_region_code'].unique()
            features = pd.DataFrame()
            targets = pd.DataFrame()

            input_seq_len = self.config.input_seq_len
            step_size = self.config.step_size

            for code in tqdm(region_codes, desc="Transforming TS Data"):
                ts_one = ts_data[ts_data['sub_region_code'] == code].sort_values(by='date')
                indices = self.get_cutoff_indices(ts_one, input_seq_len, step_size)

                x = np.zeros((len(indices), input_seq_len), dtype=np.float64)
                y = np.zeros((len(indices)), dtype=np.float64)
                date_hours, temps = [], []

                for i, (start, mid, end) in enumerate(indices):
                    x[i, :] = ts_one.iloc[start:mid]['demand'].values
                    y[i] = ts_one.iloc[mid]['demand']
                    date_hours.append(ts_one.iloc[mid]['date'])
                    temps.append(ts_one.iloc[mid]['temperature_2m'])

                features_one = pd.DataFrame(
                    x,
                    columns=[f'demand_prev_{i+1}_hr' for i in reversed(range(input_seq_len))]
                )
                features_one['date'] = date_hours
                features_one['sub_region_code'] = code
                features_one['temperature_2m'] = temps

                targets_one = pd.DataFrame(y, columns=['target_demand_next_hour'])

                features = pd.concat([features, features_one], ignore_index=True)
                targets = pd.concat([targets, targets_one], ignore_index=True)

            return features, targets['target_demand_next_hour']

    def train(self):
        train_df, test_df = self.load_inputs()
        train_x, train_y = self.generate_ts_features_and_target(train_df)
        test_x, test_y = self.generate_ts_features_and_target(test_df)
        
        train_x = train_x.drop(columns=["date"], errors="ignore")
        test_x = test_x.drop(columns=["date"], errors="ignore")

        logger.info(f'shape of train_x:{train_x.shape}, train_y:{train_y.shape}, test_x:{test_x.shape}, test_y:{test_y.shape}')

        test_x.to_parquet(self.config.x_transform)
        test_y.to_frame().to_parquet(self.config.y_transform)


        self.feature_store.upload_file_safely(self.config.x_transform, os.path.basename(self.config.x_transform))
        self.feature_store.upload_file_safely(self.config.y_transform, os.path.basename(self.config.y_transform))

        best_overall = {"model_name": None, "score": float("inf"), "params": None}

        for model_name, model_info in self.models.items():
            logger.info(f"Starting Optuna HPO for {model_name}")

            def objective(trial):
                params = model_info["search_space"](trial)
                model = model_info["class"](**params)

                x_train, x_val, y_train, y_val = train_test_split(
                    train_x, train_y, test_size=0.2, shuffle=False
                )

                model.fit(x_train, y_train)
                preds = model.predict(x_val)
                rmse = root_mean_squared_error(y_val, preds)
                return rmse

            # Create and run the study for this model
            study = optuna.create_study(direction="minimize")
            study.optimize(objective, n_trials=self.config.n_trials)

            best_params = study.best_params
            logger.info(f"Best params for {model_name}: {best_params}")

            model = model_info["class"](**best_params)
            kf = KFold(n_splits=5, shuffle=False)
            scores = cross_val_score(model, train_x, train_y, cv=kf, scoring="neg_root_mean_squared_error")
            mean_score = -scores.mean()

            with mlflow.start_run(run_name=f"{model_name}_best"):
                mlflow.log_params(best_params)
                mlflow.log_metric("cv_rmse", mean_score)
                mlflow.set_tag("model_name", model_name)

            if mean_score < best_overall["score"]:
                best_overall.update({
                    "model_name": model_name,
                    "score": mean_score,
                    "params": best_params
                })

        best_model_class = self.models[best_overall["model_name"]]["class"]
        final_params = best_overall["params"]
        best_model = best_model_class(**final_params)
        best_model.fit(train_x, train_y)

        model_path = Path(self.config.root_dir) / self.config.model_name
        create_directories([model_path.parent])
        save_bin(best_model, model_path)

        self.feature_store.upload_file_safely(model_path, "model.joblib")

        with mlflow.start_run(run_name=f"{best_overall['model_name']}_final"):
            mlflow.log_params(final_params)
            mlflow.log_metric("cv_rmse", best_overall["score"])
            mlflow.set_tag("stage", "final")

        logger.info(f"Best model: {best_overall}")
        return best_overall

In [ ]:
try:    
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    feature_store_config = config.get_feature_store_config() 
    model_trainer = ModelTrainer(config=model_trainer_config,feature_store_config=feature_store_config)
    model_trainer.train()

except Exception as e:
    raise CustomException(e, sys) from e

[2025-07-11 14:58:03,533: INFO: helpers: yaml file: config_file\config.yaml loaded successfully]
[2025-07-11 14:58:03,537: INFO: helpers: yaml file: config_file\params.yaml loaded successfully]
[2025-07-11 14:58:03,544: INFO: helpers: yaml file: config_file\schema.yaml loaded successfully]
[2025-07-11 14:58:03,548: INFO: helpers: created directory at: artifacts]
[2025-07-11 14:58:03,549: INFO: helpers: created directory at: artifacts/model_trainer]
[2025-07-11 14:58:03,553: INFO: external: Initializing external client]
[2025-07-11 14:58:03,554: INFO: external: Base URL: https://c.app.hopsworks.ai:443]
[2025-07-11 14:58:06,960: INFO: python: Python Engine initialized.]

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1237149
[2025-07-11 14:58:09,525: INFO: 468788050: Connected to Hopsworks Feature Store: JavithNaseem]
[2025-07-11 14:58:09,550: WARNING: file_store: Malformed experiment '141997247217478997'. Detailed error Yaml file '.\mlruns\141997247217478997\meta

Downloading: 0.000%|          | 0/999134 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/1025674 elapsed<00:00 remaining<?

Transforming TS Data: 100%|██████████| 11/11 [00:00<00:00, 27.36it/s]

[2025-07-11 14:58:51,848: INFO: 3080091477: shape of train_x:(1969, 674), train_y:(1969,), test_x:(2112, 674), test_y:(2112,)]


...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014922 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 171515
[LightGBM] [Info] Number of data points in the train set: 1575, number of used features: 674
[LightGBM] [Info] Start training from score 1736.014603


[I 2025-07-11 15:00:25,052] Trial 0 finished with value: 113.29673425759209 and parameters: {'num_leaves': 55, 'learning_rate': 0.21092864357878932, 'n_estimators': 240}. Best is trial 0 with value: 113.29673425759209.


[2025-07-11 15:00:25,053: INFO: 3080091477: Best params for LightGBM: {'num_leaves': 55, 'learning_rate': 0.21092864357878932, 'n_estimators': 240}]
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.011986 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 171515
[LightGBM] [Info] Number of data points in the train set: 1575, number of used features: 674
[LightGBM] [Info] Start training from score 1767.797460
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012593 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 171516
[LightGBM] [Info] Number of data points in the train set: 1575, number of used features: 674
[LightGBM] [Info] Start training from score 1812.050794
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008561 seconds.
You can set `force_col_wise=true` to remove the overh

Uploading f:\WattPredictor\artifacts\model_trainer\model.joblib: 0.000%|          | 0/1216576 elapsed<00:00 re…

[2025-07-11 15:01:46,278: INFO: 468788050: Uploaded file to Feature Store: model.joblib]
[2025-07-11 15:01:46,369: INFO: 3080091477: Best model: {'model_name': 'LightGBM', 'score': 426.507156386507, 'params': {'num_leaves': 55, 'learning_rate': 0.21092864357878932, 'n_estimators': 240}}]
